In [ ]:
import pandas as pd
import numpy as np
from scipy import stats
import seaborn as sns
import random
import math

In [ ]:
DATA_CSV = 'data/ME_clean_nh.csv'

In [ ]:
raw_points = pd.read_csv(DATA_CSV, index_col='py')

In [ ]:
prev_py = None
prev_carb = None
data_points = []
total_diff_y = 0.0
for py, row in raw_points.iterrows():
    if (prev_py != None) and (prev_py // 10000 == py // 10000):
        data_points.append([prev_carb, row['carb']])
        total_diff_y += py - prev_py
    prev_py = py
    prev_carb = row['carb']
total_diff_y /= len(data_points)

In [ ]:
df = pd.DataFrame(data_points, columns=['prev', 'post'])
g = sns.lmplot(x='prev', y='post', data=df, legend=False)
sns.plt.show(g)

In [ ]:
training = []
test = []
prob = 10
for point in data_points:
    if random.randint(1, prob) == 1:
        prob = 10
        test.append(point)
    else:
        prob -= 1
        training.append(point)

In [ ]:
training = np.array(training)

In [ ]:
m = stats.linregress(training).slope
n = stats.linregress(training).intercept

In [ ]:
sq_error = 0.0
for elem in test:
    predicted = m * elem[0] + n
    actual = elem[1]
    sq_error += (actual - predicted) ** 2
mse = math.sqrt(sq_error/len(test))